# Exploring Cartesian Product

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

#%run prelude.py
%run prelude.py --no-show-animations

## Cartesian Product

In [ ]:
#
# Function to create graph inputs
#

def create_tensors():
    a = Tensor.fromUncompressed(["M"], [0, 1, 2, 0, 0, 6, 7, 0, 9])
    b = Tensor.fromUncompressed(["N"], [0, 0, 0, 30, 0, 60, 0, 80, 90])
    z = Tensor(rank_ids=["m", "n"])
    
    a.setColor("blue")
    b.setColor("green")
    
    canvas = createCanvas(a, b, z)
    
    return (z, a, b, canvas)

## Simple Cartesian product - with ordered outputs

In [ ]:
# Cartesian product with assignment to z_ref

(z, a, b, canvas) = create_tensors()

a_m = a.getRoot()
b_n = b.getRoot()
z_m = z.getRoot()


for m, (z_n, a_val) in z_m << a_m:
    for n, (z_n_ref, b_val) in z_n << b_n:
        z_n_ref += a_val * b_val
        addFrame(canvas, [m], [n], [m,n])
        
        
displayCanvas(canvas)

## Cartesian product with outputs updated based on coordinates

This example illustrates updating outputs based on coordinates using the Fiber.getPayloadRef(&lt;coord&gt;) method. This corresponds to doing a scatter write. Whether the write is into a uncompressed or compressed space is representation dependent. 

Note that procedure isn't really necessary because the outputs are generated in a concordent order.

In [ ]:
# Cartesian product with assignment to z payload

(z, a, b, canvas) = create_tensors()

a_m = a.getRoot()
b_n = b.getRoot()
z_m = z.getRoot()


for m, (a_val) in a_m:
    for n, (b_val) in b_n:
        p = z_m.getPayloadRef(m, n)
        p += a_val * b_val
        addFrame(canvas, [m], [n], [m, n])
        
        
displayCanvas(canvas)

### Changing computation order

Note that by using Fiber.getPayloadRef() we don't need to maintain a concordant traversal order for the writes. So the following sequence with the "for" loops reversed works just fine.

In [ ]:
# Cartesian product with assignment to z payload

(z, a, b, canvas) = create_tensors()

a_m = a.getRoot()
b_n = b.getRoot()
z_m = z.getRoot()


for n, (b_val) in b_n:
    for m, (a_val) in a_m:
        p = z_m.getPayloadRef(m, n)
        p += a_val * b_val
        addFrame(canvas, [m], [n], [m, n])
        
        
displayCanvas(canvas)

## Cartesian product with position split inputs

This example splits the input vectors uniformly in postion space (into groups of 2) and illustrates the processing sequence for a 2x2 parallel Cartesian product. 

### Generate inputs

In [ ]:
# Get inputs

(z, a, b, _) = create_tensors()

a_m = a.getRoot()
b_n = b.getRoot()
z_m = z.getRoot()


### Split the input vectors

In [ ]:
# Run cartesian product

a_m1 = a_m.splitEqual(2)
print("Split a")
displayTensor(a_m1)

b_n1 = b_n.splitEqual(2)
print("Split b")
displayTensor(b_n1)

### Process the split vectors

In [ ]:
for m1, (a_m0) in a_m1:
    print(f"Process a_m0: {a_m0}")
    
    for n1,(b_n0) in b_n1:
        print(f"Process b_n0: {b_n0}")
        
        # The following two loops can be run in parallel
        for m0, a_val in a_m0:
            for n0, b_val in b_n0:
                # Note: m0 and n0 are the original coordinates
                print(f"Generate result for m0={m0}, n0={n0}")
                p = z_m.getPayloadRef(m0, n0)
                p += a_val * b_val
  
displayTensor(z)

## Testing area

For running alternative algorithms